In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random
from numpy import linalg

def k_init(X, k):
    """ k-means++: initialization algorithm

    Parameters
    ----------
    X: array, shape(n ,d)
        Input array of n samples and d features

    k: int
        The number of clusters

    Returns
    -------
    init_centers: array (k, d)
        The initialize centers for kmeans++
    """
    centroid = X.sample().values
    if k == 1:
        return centroid
    x = X.values

    for _ in range(k-1):
        dis = []
        sum_d = 0
        ind = []

        for i in range(len(x)):
            min_dis = np.inf
            for j in range(len(centroid)):
                min_dis = min(min_dis,np.linalg.norm(x[i] - centroid[j]) ** 2)

            dis.append(min_dis)
            sum_d+=min_dis

        probs = [dis[i]/sum_d for i in range(len(x))]
        for i in range(X.shape[0]):
            ind.append(i)
        next_c_ind = np.random.choice(ind,1,probs)
        next_c = X.iloc[next_c_ind].values
        centroid = np.vstack((centroid, next_c))

    return centroid
  

def assign_data2clusters(X, C):
    """ Assignments of data to the clusters
    Parameters
    ----------
    X: array, shape(n ,d)
        Input array of n samples and d features

    C: array, shape(k ,d)
        The final cluster centers

    Returns
    -------
    data_map: array, shape(n, k)
        The binary matrix A which shows the assignments of data points (X) to
        the input centers (C).
    """
    dmap = np.zeros([X.shape[0], C.shape[0]])
    for i in range(X.shape[0]):
        min_dist = float('inf')
        mincent = -1
        for j in range(C.shape[0]):
            temp_dist = np.linalg.norm(X.iloc[i, :] - C[j]) ** 2
            if temp_dist < min_dist:
                min_dist = temp_dist
                mincent = j
        dmap[i] = [1 if index == mincent else 0 for index in range(C.shape[0])]
    return dmap
        


def compute_objective(X, C):
    """ Compute the clustering objective for X and C
    Parameters
    ----------
    X: array, shape(n ,d)
        Input array of n samples and d features

    C: array, shape(k ,d)
        The final cluster centers

    Returns
    -------
    accuracy: float
        The objective for the given assigments
    """
    obj = 0
    for i in range(X.shape[0]):
        min_d = float('inf')
        for j in range(1,C.shape[0]):
            min_d = min(min_d,np.linalg.norm(X.iloc[i, :] - C[j]) ** 2)
        obj+=min_d

    return obj
        


def k_means_pp(X, k, max_iter):
    """ k-means++ clustering algorithm

    step 1: call k_init() to initialize the centers
    step 2: iteratively refine the assignments

    Parameters
    ----------
    X: array, shape(n ,d)
        Input array of n samples and d features

    k: int
        The number of clusters

    max_iter: int
        Maximum number of iteration

    Returns
    -------
    final_centers: array, shape (k, d)
        The final cluster centers
    objective_values: array, shape (max_iter)
        The objective value at each iteration
    """
    centroid = k_init(X, k)
    obj = compute_objective(X, centroid)

    for _ in range(max_iter):
        s = np.zeros([k, X.shape[1]])
        ct = np.zeros(k)
        data_map = assign_data2clusters(X, centroid)
        for i in range(X.shape[0]):
            for j in range(k):
                if data_map[i][j] == 1:
                    s[j] = s[j] + X.iloc[i, :]
                    ct[j] = ct[j] + 1

        ncentroid = np.zeros([k, X.shape[1]])
        for i in range(k):
            ncentroid[i] = s[i] / int(ct[i])

        new_obj = compute_objective(X, ncentroid)
        if new_obj < obj:
            obj = new_obj
            centroid = ncentroid


    return centroid




data = pd.read_csv("iris.data")
data.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
x1 = data['sepal_length'] / data['sepal_width']
x2 = data['petal_length'] / data['petal_width']
y = data['class']
train_data = pd.DataFrame()
train_data['x1'] = x1
train_data['x2'] = x2
for color, group in data.groupby("class"):
    plt.scatter(x1[group.index], x2[group.index], label=color)
plt.xlabel("x1 - sepal length/width")
plt.ylabel("x2 - petal length/width")
plt.show()

objective = []
for i in range(1,6):
    centers = k_means_pp(train_data, i, 50)
    objective.append(compute_objective(train_data, centers))

plt.plot([_ for _ in range(1,6)], objective)
plt.xlabel("Number of clusters")
plt.ylabel("Objective function")
plt.show()

# we choose k=3 because of steepest decline

centers = k_means_pp(train_data, 3, 50)
for c in centers:
    for color, group in data.groupby("class"):
        plt.scatter(x1[group.index], x2[group.index], label=color)
    plt.scatter(c[0], c[1], c='green', s=200, alpha=0.5)
plt.xlabel("x1 - sepal length/width")
plt.ylabel("x2 - petal length/width")
plt.show()

obj1 = []
for i in range(1,50):
    centers = k_means_pp(train_data, 3, i)
    obj1.append(compute_objective(train_data, centers))
        
plt.plot([_ for _ in range(1,50)], obj1)
plt.xlabel("Number of iterations")
plt.ylabel("Objective function")
plt.show()